# Invitations Alter Verwalter

In [1]:
# imports
import pandas as pd
import numpy as np
import datetime as dt  # dealing with dates
from random import randrange  # returns a randomly selected element from the specified range
from fpdf import FPDF  # creating pdf in python
from pdflatex import PDFLaTeX  # creating pdf from latex template
import barcode as bc  # generate barcodes
from pathlib import Path  # filesystem paths with appropriate semantics
import os  # interacting with filesystem
from docx2pdf import convert
from docx import Document

In [2]:
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(0, int_delta, 240)  # start stop step
    return start + dt.timedelta(seconds=random_second)

In [3]:
def generate_barcode(string, name_dir, save_as="barcode"):
    """
    takes input string and encodes it as barcode which is saved to the current directroy
    """
    my_code = bc.codex.Code128(str(string), writer=bc.writer.ImageWriter())
    my_code.default_writer_options['write_text'] = False
    my_code.save(f"{name_dir}/{save_as}")  # saves as png because fpdf cant handle svg

In [4]:
def generate_ticket(name, time_slot, name_dir, save_as="ticket.pdf"):
    """
    create entry ticket with barcode for input person and timeslot and saves it to given directory
    """
    # create pdf object
    pdf = FPDF('P', 'mm', 'A4')
    pdf.add_page()
    pdf.set_font('times', '', 12)

    # insert barcode
    pdf.image(f"{name_dir}/barcode.png", 100, 28, 100)

    ticket_text = "Dieses Ticket berechtigt balbabblblabala. Ticket zum unten indizierten Zeitpunkt am Einlass vorzeigen."

    # create table
    time_slot_end = time_slot + + dt.timedelta(seconds=240)
    table_data = (
        ("Veranstaltung", "Alter Verwalter!"),
        ("Gast",  name),
        ("Gültigkeitszeitraum", str(time_slot) + " - " + str(time_slot_end.time())),
        ("Veranstatungsort", "Mommsenstraße 1, 01069 Dresden")
    )

    # insert headline#
    pdf.set_font('times', '', 20)
    pdf.set_text_color(255, 0, 0)
    pdf.cell(50, 10, txt="Alter Verwalter! - Eintrittskarte", fill=False, ln=True)
    pdf.set_font('times', '', 12)
    pdf.set_text_color(0, 0, 0)
    pdf.cell(10, 10, ln=True)
    pdf.multi_cell(90, 12.5, txt=ticket_text, border=True, ln=True)
    pdf.cell(10, 10, ln=True)

    # define table
    line_height = pdf.font_size * 2.5
    col_width = pdf.epw / 2  # distribute content evenly
    for row in table_data:
        for datum in row:
            pdf.multi_cell(col_width, line_height, datum, border=1,
                    new_x="RIGHT", new_y="TOP", max_line_height=pdf.font_size)
        pdf.ln(line_height)

    pdf.output(f"{name_dir}/ticket_{name.replace(' ', '_')}.pdf")

In [5]:
def update_invitation_template(name, surname, gender, adress, template, name_dir):
    """
    updates latex template and inserts name and adress
    """
    
    # access file and replace the placeholders in the template
    if gender == "m":
        salutation = "geehrter Herr"
    elif gender == "w":
        salutation = "geehrte Frau"
    else:
        print("ungültiges Geschlecht")
        sys.exit()  
    
    # open template
    with open(template, 'r', encoding='utf8') as f:
        file = f.read() \
        .replace("<anrede>", salutation).replace("<name>", str(name)).replace("<adress>", adress) \
        .replace("<surname>", surname).replace('ü', '{\"u}').replace('ä', '{\"a}').replace('ö', '{\"o}').replace('ß', '{\"s}')

    # create updated template
    with open(f"{name_dir}/invitation_{name.replace(' ', '_')}.tex", 'w') as f2:
        f2.write(file)
        f2.close()

In [6]:
def generate_invitation(name, template, name_dir):
    """
    creates invitation from template
    """
    pdfl = PDFLaTeX.from_texfile(f'{name_dir}/{template}')
    pdf, log, completed_process = pdfl.create_pdf(keep_pdf_file=True, keep_log_file=True)
    os.replace(f"invitation_{name.replace(' ', '_')}.log" ,f"{name_dir}/invitation_{name.replace(' ', '_')}.log")
    os.replace(f"invitation_{name.replace(' ', '_')}.pdf" ,f"{name_dir}/invitation_{name.replace(' ', '_')}.pdf")

In [7]:
def generate_absence(name, adress, template, name_dir):
    """
    updating abscence template and creating pdf
    """
    doc = Document(template)
    for paragraph in doc.paragraphs:
        if "<name>" or "<adress>" or "<date>" in paragraph.text:
            paragraph.text = paragraph.text \
            .replace("<name>", name) \
            .replace("<adress>", adress)  \
            .replace("<date>", str(dt.date.today().strftime("%d. %B %Y")))
            
    doc.save(f"{name_dir}/absence_{name.replace(' ', '_')}.docx")
    
    # update template
    #with open(template, 'r', encoding="windows-1252") as f:
     #   file = f.read() \
      #  .replace("<name>", name) \
       # .replace("<date>", str(dt.date)) \
        #.replace("<adress>", adress)
        
    # create updated template
    #with open(f"{name_dir}/absence_{name.replace(' ', '_')}.docx", 'w') as f2:
     #   f2.write(file)
      #  f2.close()
    ################################################ error here #######################################################
    # Converting docx specifying both the input and output paths
    convert(f"{name_dir}/absence_{name.replace(' ', '_')}.docx", f"{name_dir}/absage_{name.replace(' ', '_')}.pdf")  # output name must differ from input name

### Create dataframe from .csv file

In [8]:
# read from local .csv-file
df = pd.read_csv('teilnahmeliste.csv')
# dataframe vervollständigen
d1 = dt.datetime.strptime('28/1/2023 7:00 PM', '%d/%m/%Y %I:%M %p')
d2 = dt.datetime.strptime('28/1/2023 8:00 PM', '%d/%m/%Y %I:%M %p')
df["Name"]=df["Vorname"] + " " + df["Nachname"]
df["Zeitslot"]=df.apply(lambda row: random_date(d1, d2), axis=1)  # give everyone their timeslot
df.head()

,Vorname,Nachname,Adresse,Geschlecht,Wo drucken?,Zeitslot,Name
0,Freida,Nurgül,"Leopoldstraße 37, 12161 Berlin",w,Dresden,2023-01-28 19:44:00,Freida Nurgül
1,Gladys,Ronan,"Sömmeringstr. 95, 89260 Weißenhorn",m,Dresden,2023-01-28 19:56:00,Gladys Ronan
2,Anil,Ina,"Flughafenstrasse 94, 92526 Oberviechtach",w,Leipzig,2023-01-28 19:28:00,Anil Ina
3,Hjörtur,Sargis,"C/ Eras 60, 2D, 28722 El Vellón, Spain",m,Dresden,2023-01-28 19:48:00,Hjörtur Sargis
4,Lungile,Haig,"14 boulevard Albin Durand, 95000 Cergy, France",w,Marseille,2023-01-28 19:56:00,Lungile Haig


### Create folder hierarchy and call the functions for each guest

In [9]:
# create subfolder for every entry
# TODO: create barcode only if not already existing?; check if it is consistent with timeslot (aka control)
for i in df.index:
    
    # set all variables
    # read from dataframe
    name = df["Name"][i]
    surname = df["Nachname"][i]
    time_slot = df["Zeitslot"][i]
    gender = df["Geschlecht"][i]
    adress_tex = df["Adresse"][i].replace(",", "\\\\")  # lololololol! to recieve \\ we have to escape it 
    adress_word = df["Adresse"][i].replace(", ", "\n")
    # to create dirs
    folder_name = (df["Name"][i]).replace(" ", "_")
    name_dir = f"./einladungen/{folder_name}"
    # to update ticket
    tex_template = "brief.tex"
    # to update absence
    absence_template = "fernbleiben.docx"
    
    # create directories
    Path(f"./einladungen/{folder_name}").mkdir(parents=True, exist_ok=True)  # create subfolder if not existing
    nmae_dir = f"./einladungen/{folder_name}"
    
    # create barcodecode
    generate_barcode(time_slot, name_dir)
    
    # create ticket
    generate_ticket(name, time_slot, name_dir)
    
    # update .tex template and write to local folder
    update_invitation_template(name, surname, gender, adress_tex, tex_template, name_dir)
    
    # generate invitation
    invitation_updated = f"invitation_{(df['Name'][i]).replace(' ', '_')}.tex"
    generate_invitation(name, invitation_updated, name_dir)
    
    # update .docx template and generate absence pdf
    generate_absence(name, adress_word, absence_template, name_dir)

C:\Users\Padman\AppData\Local\Temp\ipykernel_13760\3035259146.py:27: DeprecationWarning: The parameter "ln" is deprecated. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(50, 10, txt="Alter Verwalter! - Eintrittskarte", fill=False, ln=True)
C:\Users\Padman\AppData\Local\Temp\ipykernel_13760\3035259146.py:30: DeprecationWarning: The parameter "ln" is deprecated. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(10, 10, ln=True)
C:\Users\Padman\AppData\Local\Temp\ipykernel_13760\3035259146.py:31: DeprecationWarning: The parameter "ln" is deprecated. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.multi_cell(90, 12.5, txt=ticket_text, border=True, ln=True)
C:\Users\Padman\AppData\Local\Temp\ipykernel_13760\3035259146.py:32: DeprecationWarning: The parameter "ln" is deprecated. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(10, 10, ln=True)


PermissionError: [Errno 13] Permission denied: './einladungen/Freida_Nurgül/absence_Freida_Nurgül.docx'